In [1]:
# 参数
top = 200

# /media/Data/qipeng/modified_complete_images/AMNet-Rumor/AMNet-Predict/expt1.2/att_maps/rumor
server_path = '/media/Data/qipeng/modified_complete_images/AMNet-Rumor/AMNet-Predict/'
client_path = '/Users/snow/snow_学习/4-研究生/实验室/AMNet/AMNet/att_maps/'

expts = []

# for i in range(5, 7):
#     expts.append('expt' + str(i))
#     expts.append('expt' + str(i) + '.2')

# AUC 检测效果最高的三个，取大量数据
expts.append('expt2')
expts.append('expt3.2')
expts.append('expt6')

print(expts)

['expt2', 'expt3.2', 'expt6']


In [2]:
import pandas as pd
import os
import time
    
for expt in expts:
    eval_file = '../predict/eval-' + expt + '.csv'
    df = pd.read_csv(eval_file)
    
    # 得到四类图片的文件列表
    
    rumor_bottom_list = df[df['Label'].isin([1])].sort_values(by='PredValue')[:top]['ImgName'].tolist()
    rumor_top_list = df[df['Label'].isin([1])].sort_values(by='PredValue', ascending=False)[:top]['ImgName'].tolist()
    nonrumor_bottom_list = df[df['Label'].isin([0])].sort_values(by='PredValue')[:top]['ImgName'].tolist()
    nonrumor_top_list = df[df['Label'].isin([0])].sort_values(by='PredValue', ascending=False)[:top]['ImgName'].tolist()
    
    expt_file_dict = {}
    expt_file_dict['rumor-bottom'] = rumor_bottom_list
    expt_file_dict['rumor-top'] = rumor_top_list
    expt_file_dict['nonrumor-bottom'] = nonrumor_bottom_list
    expt_file_dict['nonrumor-top'] = nonrumor_top_list
    
    # 开始进行文件传输
    
    rumor_path = server_path + expt + '/att_maps/rumor/'
    nonrumor_path = server_path + expt + '/att_maps/nonrumor/'
    des_path = client_path + expt + '/'
    
    for i in range(top):
        for preffix, name_list in expt_file_dict.items():
            img_name = name_list[i].split('.jpg')[0] + '_att.jpg'
            
            if 'nonrumor' not in preffix:
                src_file = rumor_path + img_name
            else:
                src_file = nonrumor_path + img_name
                
            des_file = des_path + preffix + str(i + 1) + '-' + img_name
            
            # 执行expect脚本文件，把src_file传给des_file
            # 命令：expect <.sh> <src_file> <des_file> 
            os.system('expect scp.sh ' + src_file + ' ' + des_file)